In [1]:
# ! pip install qdrant-client

## Generatring  splade vectors

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_id = 'naver/splade-cocondenser-ensembledistil'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

In [3]:
text = """Arthur Robert Ashe Jr. (July 10, 1943 – February 6, 1993) was an American professional tennis player. He won three Grand Slam titles in singles and two in doubles."""


In [4]:
import torch

def compute_vector(text):
    """
    Computes a vector from logits and attention mask using ReLU, log, and max operations.

    Args:
    logits (torch.Tensor): The logits output from a model.
    attention_mask (torch.Tensor): The attention mask corresponding to the input tokens.

    Returns:
    torch.Tensor: Computed vector.
    """
    tokens = tokenizer(text, return_tensors="pt")
    output = model(**tokens)
    logits, attention_mask = output.logits, tokens.attention_mask
    relu_log = torch.log(1 + torch.relu(logits))
    weighted_log = relu_log * attention_mask.unsqueeze(-1)
    max_val, _ = torch.max(weighted_log, dim=1)
    vec = max_val.squeeze()

    return vec, tokens


vec, tokens = compute_vector(text)
print(vec.shape)


torch.Size([30522])


In [5]:
def extract_and_map_sparse_vector(vector, tokenizer):
    """
    Extracts non-zero elements from a given vector and maps these elements to their human-readable tokens using a tokenizer. The function creates and returns a sorted dictionary where keys are the tokens corresponding to non-zero elements in the vector, and values are the weights of these elements, sorted in descending order of weights.

    This function is useful in NLP tasks where you need to understand the significance of different tokens based on a model's output vector. It first identifies non-zero values in the vector, maps them to tokens, and sorts them by weight for better interpretability.

    Args:
    vector (torch.Tensor): A PyTorch tensor from which to extract non-zero elements.
    tokenizer: The tokenizer used for tokenization in the model, providing the mapping from tokens to indices.

    Returns:
    dict: A sorted dictionary mapping human-readable tokens to their corresponding non-zero weights.
    """

    # Extract indices and values of non-zero elements in the vector
    cols = vector.nonzero().squeeze().cpu().tolist()
    weights = vector[cols].cpu().tolist()

    # Map indices to tokens and create a dictionary
    idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}
    token_weight_dict = {idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)}

    # Sort the dictionary by weights in descending order
    sorted_token_weight_dict = {k: v for k, v in sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True)}

    return sorted_token_weight_dict

# Usage example
sorted_tokens = extract_and_map_sparse_vector(vec, tokenizer)
sorted_tokens

{'ashe': 2.95,
 'arthur': 2.61,
 'tennis': 2.22,
 'robert': 1.74,
 'jr': 1.55,
 'he': 1.39,
 'founder': 1.36,
 'doubles': 1.24,
 'won': 1.22,
 'slam': 1.22,
 'died': 1.19,
 'singles': 1.1,
 'was': 1.07,
 'player': 1.06,
 'titles': 0.99,
 'birthday': 0.99,
 'grand': 0.93,
 'champion': 0.93,
 'many': 0.92,
 'actor': 0.92,
 'his': 0.89,
 'death': 0.89,
 '1943': 0.86,
 'merlin': 0.86,
 'birth': 0.85,
 'date': 0.84,
 'win': 0.83,
 'professional': 0.81,
 '1993': 0.79,
 'early': 0.78,
 'born': 0.77,
 'medal': 0.75,
 'double': 0.73,
 'sr': 0.73,
 'championship': 0.71,
 'played': 0.59,
 'celebrity': 0.59,
 'nationality': 0.59,
 'wimbledon': 0.53,
 'championships': 0.5,
 'williams': 0.5,
 'son': 0.49,
 'carter': 0.49,
 'tournament': 0.44,
 'title': 0.43,
 '1991': 0.43,
 'champ': 0.43,
 'edgar': 0.4,
 'alex': 0.38,
 'medals': 0.37,
 'him': 0.35,
 'career': 0.35,
 'hale': 0.33,
 'bob': 0.32,
 'nathan': 0.31,
 'players': 0.3,
 'retired': 0.29,
 'slams': 0.29,
 'award': 0.26,
 'gilbert': 0.26,
 'had

In [12]:
# Qdrant client setup
from qdrant_client import QdrantClient
from qdrant_client import models

# Define collection name
COLLECTION_NAME = "example_collection"

# Insert sparse vector into Qdrant collection
point_id = 1  # Assign a unique ID for the point

client = QdrantClient("http://localhost:6333")

In [13]:

client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={},
    sparse_vectors_config={
        "text": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

## insert vectors to collection

In [14]:
import json
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from tqdm.auto import tqdm

FILE = "../data/data.json"
metadata_keys = ["name", "summary", "url", "category", "updated_at"]
workplace_docs = []

# load documents from source to langchain doc store
with open(FILE, "rt") as f:
    for doc in json.loads(f.read()):
        workplace_docs.append(
            Document(
                page_content=doc["content"],
                metadata={k: doc.get(k) for k in metadata_keys},
            )
        )

print(f"Loaded {len(workplace_docs)} documents")


# split documents based on splitter
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer=tokenizer,
    chunk_size=512, chunk_overlap=256
)
docs = text_splitter.transform_documents(workplace_docs)

# print(docs[0])
# print(len(docs))

for doc in tqdm(docs):
    vec, tokens = compute_vector(doc.page_content)
    
    indices = vec.nonzero().numpy().flatten()
    values = vec.detach().numpy()[indices]
    
    point_id = str(uuid.uuid4())
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(
                id=point_id,
                payload={key: doc.to_json()["kwargs"]["metadata"].get(key) for key in metadata_keys},  # Add any additional payload if necessary
                vector={
                    "text": models.SparseVector(
                        indices=indices.tolist(), values=values.tolist()
                    )
                },
            )
        ],
    )

Loaded 15 documents


  0%|          | 0/28 [00:00<?, ?it/s]

## find similar text via vector search

In [22]:
# Preparing a query vector

query_text = "What are companies views about WFH?"
query_vec, query_tokens = compute_vector(query_text)
query_vec.shape

query_expansion = extract_and_map_sparse_vector(query_vec, tokenizer)
print(query_expansion)

query_indices = query_vec.nonzero().numpy().flatten()
query_values = query_vec.detach().numpy()[query_indices]

# Searching for similar documents
result = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="text",
        vector=models.SparseVector(
            indices=query_indices,
            values=query_values,
        ),
    ),
    with_vectors=True,
)

print([res.payload["name"] for res in result])
print(len(result))

{'w': 2.51, '##f': 2.34, 'view': 2.08, '##h': 2.05, 'views': 2.02, 'companies': 1.88, 'company': 1.65, 'businesses': 0.97, 'industry': 0.59, 'about': 0.46, 'success': 0.41, 'marketing': 0.39, 'industries': 0.37, 'corporate': 0.31, 'stock': 0.3, 'benefit': 0.29, 'association': 0.24, 'strategy': 0.23, 'market': 0.2, 'offer': 0.2, 'investment': 0.18, 'manufacturer': 0.17, 'competitor': 0.17, 'rating': 0.16, 'advertising': 0.15, 'said': 0.11, 'useful': 0.09, 'organization': 0.08, '?': 0.06, 'brand': 0.06, 'trade': 0.01, 'product': 0.01, 'website': 0.01}
['Fy2024 Company Sales Strategy', 'New Employee Onboarding Guide', 'Intellectual Property Policy', 'Fy2024 Company Sales Strategy', 'Compensation Framework For It Teams', 'Sales Engineering Collaboration', 'Sales Organization Overview', 'Compensation Framework For It Teams', 'Code Of Conduct', 'April Work From Home Update']
10


# Ask llm